In [ ]:
! make clean

In [ ]:
! make clean-logs

In [ ]:
# ! rm  -rf /media/sayem/510B93E12554BBD1/Hangman/wandb
# ! rm -rf /media/sayem/510B93E12554BBD1/checkpoints

In [ ]:
import torch
import numpy as np
import random
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / "data"
DATASET_DIR = PROJECT_ROOT / "dataset"

from dataset.data_generation import read_words_list


def set_seed(seed):
    """Set seed for reproducibility."""
    random.seed(seed)  # Python random module
    np.random.seed(seed)  # Numpy module
    torch.manual_seed(seed)  # PyTorch
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)  # Sets seed for CUDA (GPU)
        torch.cuda.manual_seed_all(seed)  # Ensure reproducibility on all GPUs
        torch.backends.cudnn.deterministic = True  # Use deterministic algorithms
        torch.backends.cudnn.benchmark = (
            False  # If input sizes do not vary, this should be set to False
        )


# Example usage:
set_seed(42)  # Use any number to seed all libraries

#### Imports


In [ ]:
corpus_path = DATA_DIR / "words_250000_train.txt"
corpus = read_words_list(corpus_path)
len(corpus)

### API


In [ ]:
from api.hangman_api import HangmanAPI
import time

In [ ]:
import os

env_path = PROJECT_ROOT / ".env"
api_key = None
if env_path.exists():
    with env_path.open() as env_file:
        for raw_line in env_file:
            line = raw_line.strip()
            if not line or line.startswith("#"):
                continue
            key, _, value = line.partition("=")
            if key.strip() == "API":
                value = value.strip()
                value = value.strip('"')
                value = value.strip("'")
                api_key = value
                break

if not api_key:
    raise RuntimeError("API key not found in .env")

os.environ["API"] = api_key

In [ ]:
from functools import partial
import logging

from api.guess_strategies import (
    neural_guess_strategy,
    frequency_guess_strategy,
    ngram_guess_strategy,
)
from dataset.encoder_utils import DEFAULT_ALPHABET
from models import (
    HangmanBiLSTM,
    HangmanBiLSTMConfig,
    HangmanLightningModule,
    HangmanTransformer,
    HangmanTransformerConfig,
    TrainingModuleConfig,
)

logger = logging.getLogger("hangman.notebook")
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.INFO)
    logger.addHandler(handler)
logger.setLevel(logging.INFO)


def load_neural_strategy(checkpoint_dir: Path) -> tuple[callable, str]:
    checkpoint_files = sorted(checkpoint_dir.glob("best-hangman-*.ckpt"))
    if not checkpoint_files:
        logger.warning(
            "No checkpoints found in %s. Falling back to frequency strategy.",
            checkpoint_dir,
        )
        return frequency_guess_strategy, "frequency"

    def _score(path: Path) -> float:
        try:
            return float(path.stem.split("=")[-1])
        except ValueError:
            return float("-inf")

    best_checkpoint = max(checkpoint_files, key=_score)
    logger.info("Loading checkpoint: %s", best_checkpoint.name)

    checkpoint = torch.load(best_checkpoint, map_location="cpu")

    model_class_name = "HangmanBiLSTM"
    hyper_params = checkpoint.get("hyper_parameters")
    if isinstance(hyper_params, dict):
        model_class_name = hyper_params.get("model_class", model_class_name)

    vocab_size = len(DEFAULT_ALPHABET)
    mask_idx = len(DEFAULT_ALPHABET)
    pad_idx = len(DEFAULT_ALPHABET) + 1

    if "Transformer" in model_class_name:
        model_config = HangmanTransformerConfig(
            vocab_size=vocab_size,
            mask_idx=mask_idx,
            pad_idx=pad_idx,
            max_word_length=45,
        )
        base_model = HangmanTransformer(model_config)
    else:
        model_config = HangmanBiLSTMConfig(
            vocab_size=vocab_size,
            mask_idx=mask_idx,
            pad_idx=pad_idx,
        )
        base_model = HangmanBiLSTM(model_config)

    lightning_module = HangmanLightningModule(base_model, TrainingModuleConfig())
    lightning_module.load_state_dict(checkpoint["state_dict"])
    lightning_module.eval()

    return (
        partial(neural_guess_strategy, model=lightning_module.model),
        best_checkpoint.name,
    )


dictionary_path = (DATA_DIR / "words_250000_train.txt").resolve()
checkpoint_dir = (PROJECT_ROOT / "logs" / "checkpoints").resolve()

# try:
strategy_callable, checkpoint_name = load_neural_strategy(checkpoint_dir)
# if checkpoint_name != "frequency":
#     logger.info("Using neural strategy from %s", checkpoint_name)
# else:
#     logger.info("Using fallback frequency strategy")
# except Exception as exc:  # noqa: BLE001 - display to notebook
# logger.exception(
#     "Failed to load neural strategy, falling back to frequency strategy"
# )
# strategy_callable = frequency_guess_strategy

api = HangmanAPI(
    access_token=api_key,
    dict_path=str(dictionary_path),
    strategy=strategy_callable,
)

In [ ]:
# Testing a single game
api.start_game(practice=1, verbose=True)
[
    total_practice_runs,
    total_recorded_runs,
    total_recorded_successes,
    total_practice_successes,
] = api.my_status()  # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print(
    "run %d practice games out of an allotted 100,000. practice success rate so far = %.3f"
    % (total_practice_runs, practice_success_rate)
)

In [ ]:
# Status before practice run
before_status = api.my_status()
(
    before_practice_runs,
    before_recorded_runs,
    before_recorded_successes,
    before_practice_successes,
) = before_status

before_practice_win_rate = (
    before_practice_successes / before_practice_runs if before_practice_runs else 0.0
)

print(f"Before practice runs: {before_practice_runs}")
print(f"Before practice successes: {before_practice_successes}")
print(f"Before recorded runs: {before_recorded_runs}")
print(f"Before recorded successes: {before_recorded_successes}")
print(f"Practice win rate so far: {before_practice_win_rate:.2%}")

In [ ]:
# test run 1000 practice games
from tqdm import tqdm

practice_games = 1_0
for game_index in tqdm(range(practice_games), desc="Practice games", unit="game"):
    api.start_game(practice=1, verbose=False)
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

In [ ]:
# after 1000 practice games
after_status = api.my_status()
(
    after_practice_runs,
    after_recorded_runs,
    after_recorded_successes,
    after_practice_successes,
) = after_status

after_practice_win_rate = (
    after_practice_successes / after_practice_runs if after_practice_runs else 0.0
)

print(f"After practice runs: {after_practice_runs}")
print(f"After practice successes: {after_practice_successes}")
print(f"After recorded runs: {after_recorded_runs}")
print(f"After recorded successes: {after_recorded_successes}")
print(f"Practice win rate so far: {after_practice_win_rate:.2%}")

In [ ]:
# Session performance
practice_runs_delta = after_practice_runs - before_practice_runs
practice_successes_delta = after_practice_successes - before_practice_successes
recorded_runs_delta = after_recorded_runs - before_recorded_runs
recorded_successes_delta = after_recorded_successes - before_recorded_successes

session_practice_win_rate = (
    practice_successes_delta / practice_runs_delta
    if practice_runs_delta
    else float("nan")
)
print("Session performance:")
print(f"Δ practice runs: {practice_runs_delta}")
print(f"Δ practice successes: {practice_successes_delta}")
print(f"Δ recorded runs: {recorded_runs_delta}")
print(f"Δ recorded successes: {recorded_successes_delta}")
if practice_runs_delta:
    print(f"Session practice win rate: {session_practice_win_rate:.2%}")
else:
    print("Session practice win rate: n/a (no new practice games)")

In [ ]:
# Session performance summary computed above

In [ ]:
STOP

## Playing recorded games:

Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.


In [ ]:
# test run 1000 practice games
from tqdm import tqdm

practice_games = 1_000
for game_index in tqdm(range(practice_games), desc="Practice games", unit="game"):
    # api.start_game(practice=0, verbose=False)
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

In [ ]:
[
    total_practice_runs,
    total_recorded_runs,
    total_recorded_successes,
    total_practice_successes,
] = api.my_status()  # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes / total_recorded_runs
print("overall success rate = %.3f" % success_rate)